In [1]:
import MeCab
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import pymysql
import sqlalchemy
import re

In [2]:
data0 = pd.read_csv('url_dataset_2021_drop.csv')
data = pd.read_csv('text_url_2021.csv')
data['noun'] = np.nan
data['press'] = data0['press']
data['date'] = data0['time']
data['word_list'] = np.nan
data = data[['title', 'url', 'press', 'date', 'content', 'noun']]   
data = data.astype('object')

In [3]:
data.head()

,title,url,press,date,content,noun
0,"올해는 ESG 정보공시 표준화 원년, 압박 더 거세진다",https://news.naver.com/main/read.naver?mode=LS...,머니투데이,2021.01.01.,\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,NaN
1,"서학개미 사로잡은 美 ESG ETF, 올해도 高高할까",https://news.naver.com/main/read.naver?mode=LS...,매일경제,2021.01.01.,\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,NaN
2,"[신년사]방문규 수은 행장 ""수출 6000억달러 선봉장될 것""",https://news.naver.com/main/read.naver?mode=LS...,뉴시스,2021.01.01.,\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,NaN
3,[韓산업 대전환 ①] 격변기의 시대…대전환 앞당겼다,https://news.naver.com/main/read.naver?mode=LS...,아이뉴스24,2021.01.01.,\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,NaN
4,"펀드사태 혼쭐난 금융권, 외부 `포청천` 영입",https://news.naver.com/main/read.naver?mode=LS...,매일경제,2021.01.01.,\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,NaN


In [4]:
#db 입력
conn = pymysql.connect(host='127.0.0.1', user='root', password='1234',db='esg_db', charset='utf8mb4')
cursor = conn.cursor()

In [5]:
# 공백 제거
def remove_space(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'^\s+', '', text)
    text = re.sub(r'\s+$', '', text)
    text = re.sub(r'\t', ' ', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\r', ' ', text)
    return text
    
# 따옴표 제거
def remove_quote(text):
    text = re.sub(r'\'', '', text)
    text = re.sub(r'\`', '', text)
    text = re.sub(r'\‘', '', text)
    text = re.sub(r'\’', '', text)

    text = re.sub(r'\"', '', text)
    text = re.sub(r'\“', '', text)
    text = re.sub(r'\”', '', text)
    return text

# () 괄호 안 제거 
def remove_between_round_brackets(text):
    return re.sub(r'\([^)]*\)', '', text)          

# {} 괄호 안 제거 
def remove_between_curly_brackets(text):
    return re.sub(r'\{[^}]*\}', '', text)

# [] 괄호 안 제거 
def remove_between_square_brackets(text):
    return re.sub(r'\[[^]]*\]', '', text)  

# <> 괄호 안 제거 
def remove_between_angle_brackets(text):
    return re.sub(r'\<[^>]*\>', '', text)
    
# url 제거
def remove_url(text):
    text = re.sub(r'http[s]?://(?:[\t\n\r\f\v]|[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text) 
    text = re.sub(r'[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{2,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)', '', text) 
    return text

# 이메일 제거
def remove_email(text):
    return re.sub(r'[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+','', text)        
    
# 날짜 제거
def remove_date(text):
    return re.sub(r'\d+[.]\d+[.]\d+','', text)

# 숫자 이용 문자 제거
def remove_number_text(text):
    text = re.sub(r'[0-9]+%', '', text)
    text = re.sub(r'[0-9]+층', '', text)
    return text

# 특수문자 제거    
def remove_symbol(text):
    return re.sub(r'[^\w\s]', '', text)

# 자음, 모음 제거
def remove_consonants_vowels(text):
    text = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+', '', text)
    return text

# 앞에 날리기
def remove_to_callback(news):
    callback_list = []
    try : 
        for b in re.finditer('flash_removeCallback()',news):
            callback_list.append(b.end())
        news = news[callback_list[0]:]
    except :
        news = news
    return news

# 뒤에 날리기
def remove_from_back_third_triangle(news):
    triangle_list = []
    for a in re.finditer('▶',news):
        triangle_list.append(a.start())
    if len(triangle_list) >= 3:
        news = news[:triangle_list[-3]]
    elif len(triangle_list) == 2:
        news = news[:triangle_list[-2]]
    elif len(triangle_list) == 1:
        news = news[:triangle_list[-1]]
    return news

In [6]:
# 모든 것을 한 번에 정리해서 쫙 해줌
def news_cleansing(txt):
    txt = remove_to_callback(txt)
    txt = remove_from_back_third_triangle(txt)
    txt = remove_email(txt)
    #news = remove_symbol(news)
    txt = remove_space(txt)
    txt = remove_quote(txt)
    txt = remove_url(txt)
    txt = remove_number_text(txt)
    txt = remove_consonants_vowels(txt)
    txt = remove_between_curly_brackets(txt)
    txt = remove_between_square_brackets(txt)
    txt = remove_between_angle_brackets(txt)
    return txt

In [7]:
dd = data.iloc[546,4]
news_cleansing(dd)

'()  신한퓨처스랩 7-1기 선발NH디지털 챌린지 플러스(Challenge+) 5기 모집 국내 주요 금융그룹들이 스타트업 지원에 팔을 걷어 붙였다.신한금융그룹은 최근 국내 금융권 최초 스타트업 엑셀러레이터 프로그램인 신한퓨처스랩 7-1기 32개사를 선발했다.신한퓨처스랩은 2015년 5월 1기 출범 이후 작년 6-2기까지 총 214개의 혁신적인 스타트업을 발굴 및 육성했다. 현재까지 국내·외 육성기업에 359억7000만원을 직·간접 투자했다.이번 7-1기 모집에는 총 622개 스타트업이 지원했다. 핀테크, 딥테크, 컨텐츠, 헬스케어 등 기존의 모집 분야에 더해 신한금융이 전사적으로 추진하는 ESG 경영 전략에 맞춰 ESG 스타트업도 함께 모집했다.특히 3D 아바타 SNS 기술을 보유한 로콘, AI·빅데이터 기반 통합모빌리티 서비스를 제공하는 네이처모빌리티, 마이데이터 기반 1인 마켓플레이스 플랫폼 에스앤피랩, 한의학 관련 스마트 헬스케어 O2O솔루션인 픽플스 등 최신 시장 트렌드를 반영한 스타트업과 폐플라스틱·제철 부산물을 활용해 건축·토목제품을 개발하는 이옴텍 등 ESG 분야의 스타트업이 선발됐다.NH농협은행은 혁신기업 육성·협업 프로그램 NH디지털 챌린지 플러스(Challenge+) 5기를 모집한다.NH디지털 챌린지 플러스는 디지털 혁신기업이 사업모델을 구체화하고 성장을 가속화할 수 있도록 투자·법률·마케팅·재무 분야 등의 컨설팅은 물론 범농협 계열사와의 사업제휴 기회 및 후속 투자기회 등을 제공하는 전문 액셀러레이팅 프로그램이다.이번 기수 테마는 NH와 꿈꾸다로 진행된다. 인공지능(AI), 데이터, 클라우드, 블록체인 등 신기술 뿐 아니라 마이데이터, 플랫폼서비스, 헬스·인슈어테크, 농업, ESG 등 다양한 분야에서 디지털 혁신과 범농협 협업 시너지 창출이 가능한 기업이면 누구나 참여 가능하다. 농협은행은 ▷혁신성 ▷사업성 ▷시너지 등을 종합적으로 평가해 선정할 계획이다.선정된 기업에게는 ▷NH디지털혁신캠퍼스내 업무공간 ▷역량강화 전문교육 ▷밋업데이(협업

In [8]:
# 모든 데이터 클렌징
for index in range(len(data)):
    #print(index)
    news_txt = data.loc[index, 'content']
    new_txt = news_cleansing(news_txt)
    data.loc[index,'content'] = new_txt
data.head(10)

,title,url,press,date,content,noun
0,"올해는 ESG 정보공시 표준화 원년, 압박 더 거세진다",https://news.naver.com/main/read.naver?mode=LS...,머니투데이,2021.01.01.,() 산업·금융·법조계 ESG 전문가 대담]/사진제공=이미지투데이2020년은 E...,NaN
1,"서학개미 사로잡은 美 ESG ETF, 올해도 高高할까",https://news.naver.com/main/read.naver?mode=LS...,매일경제,2021.01.01.,() 작년 착한 투자에 30조 몰려인베스코솔라 年 급등美기업 IPO 열기도 뜨거...,NaN
2,"[신년사]방문규 수은 행장 ""수출 6000억달러 선봉장될 것""",https://news.naver.com/main/read.naver?mode=LS...,뉴시스,2021.01.01.,() ESG 경영 강화...수출 중소중견기업 맞춤형 지원 방문규 수출입은행장 (사...,NaN
3,[韓산업 대전환 ①] 격변기의 시대…대전환 앞당겼다,https://news.naver.com/main/read.naver?mode=LS...,아이뉴스24,2021.01.01.,() 코로나 팬데믹에 뉴노멀 시대 활짝…대내외 경제 불안 속 미래 먹거리 찾기 총...,NaN
4,"펀드사태 혼쭐난 금융권, 외부 `포청천` 영입",https://news.naver.com/main/read.naver?mode=LS...,매일경제,2021.01.01.,"() 4대 금융지주 인사 키워드는우리금융, 감사원 출신 신민철하나銀은 이인영 변호...",NaN
5,"[TF초점] ""쉴 틈 없다"" 2021년 4대 그룹 총수 앞에 놓인 과제는",https://news.naver.com/main/read.naver?mode=LS...,더팩트,2021.01.01.,() 이재용 삼성전자 부회장은 지난달 30일 서울고법 형사1부 심리로 진행된 국정...,NaN
6,"[2021 DD 신년사] IT산업, 역동적으로 대한민국 이끌기를",https://news.naver.com/main/read.naver?mode=LS...,디지털데일리,2021.01.01.,() 2021년1월1일 오전 7시40분 울산 간절곶 해돋이 생중계 캡쳐존경하는 독...,NaN
7,신임 농협은행장에 권준학 농협중앙회 상무,https://news.naver.com/main/read.naver?mode=LS...,국민일보,2021.01.01.,() 차기 농협은행장에 권준학(57·사진) 농협중앙회 기획조정본부장(상무)이 선임...,NaN
8,[새해증시]'코스피 3000-코스닥1000' 올해 상반기 돌파한다,https://news.naver.com/main/read.naver?mode=LS...,머니투데이,2021.01.01.,"() 삼성전자 등 반도체 유망. 여유자금은 주식, 부동산, ESG 펀드에 ] 증...",NaN
9,새해 돈 있으면 부동산보다 주식…2배 이상 추천 많았다,https://news.naver.com/main/read.naver?mode=LS...,머니투데이,2021.01.01.,"() 달러, 채권 등 안전자산 인기 시들]금융투자업계 전문가들은 올해도 증시 강...",NaN


In [9]:
# 이건 뭔가...
def preproc(words):
    w2 = []
    for i, t in enumerate(words):
        w, p = t
        w=w.replace('"','\\quot')
        w=w.replace('ᆫ', 'ㄴ')
        w=w.replace('ᆯ', 'ㄹ')
        w=w.replace('ᆷ', 'ㅁ')
        w=w.replace('ᆸ', 'ㅂ')

        p=p.replace('NNBC', 'NNB') # 세종품사태그로 변환
        p=p.replace('SY', 'SO')
        p=p.replace('SC', 'SP')
        w2.append((w, p))
    return w2

#형태소 추출 코드
def posTag(text):
    tagger = MeCab.Tagger()
    s = tagger.parse(text)
    l = s.split('\n')
    pos=[]
    for item in l:
        if item=='EOS':
            break
        i2 = item.split('\t')
        i3 = i2[1].split(',')
        #print(i3)

        if '+' in i3[0]:
            t1=i3[5]
            t2=i3[6]
            w1=i3[7].split('/')[0]
            w2=(i3[7].split('+')[1]).split('/')[0]

            i = (w1, t1)
            pos.append(i)
            i = (w2, t2)
            pos.append(i)
        else:
            i = (i2[0], i3[0])
            pos.append(i)

            
    return pos

#??
def phraseTag(sent, subLevel=False, debug=False):
    tagged = posTag(sent)
    tagged = preproc(tagged)
    #print(tagged)
    posList=[]
    for token in tagged:
        posList.append(token[0]+'/'+token[1])

    #print(posList)
    return posList

In [10]:
#불용어사전
stop_words = ['사진제공', '무단', '전재', '배포', '특종', '기자', 
              '증권부장', '무단전재', '재배포', '뉴스', '기사', '구독신청', '머니', '투데이', '리얼타임', '부장', '뉴시스', '운세', '실검', 
              '데일리' '뉘', '우', '스', '그동안', '생방송', 'com', 'kr', 'newsis', 'yesphoto', 'www', '클릭', '신문', 'co', 'seoul', 'mk'
              #언론사
              '머니투데이', '매일경제', '뉴시스', '아이뉴스24', '더팩트', '디지털데일리', '국민일보', '한경비즈니스', '일간스포츠', '파이낸셜뉴스', 
              '한국경제', '서울신문', '이데일리', '전자신문', '세계일보', '연합뉴스', 'KBS', 'ZDNet Korea', '헤럴드경제', '머니S', '매경이코노미', 
              '서울경제', 'MBN', '아시아경제', '블로터', '조선일보', '뉴스1', 'SBS Biz', '데일리안', '경향신문', '중앙일보', '조세일보', '조선비즈', '스포츠동아', 
              '한국경제TV', '노컷뉴스', '디지털타임스', 'YTN', '스포츠조선', '코리아헤럴드', '문화일보', '중앙SUNDAY', '코리아중앙데일리', '스포츠서울', 'MBC', '동아일보', 
              '한국일보', '부산일보', 'SBS', '오마이뉴스', '연합뉴스TV', '한겨레', '스타뉴스', '프레시안', '주간동아', 
              '신동아', '주간조선', '이코노미스트', '인벤', '여성신문', 'TV조선', '동아사이언스', '비즈니스워치', '스포츠경향', '매일신문', 
              'OSEN', '스포츠월드', 'JTBC GOLF', 'MK스포츠', '인터풋볼', '스포탈코리아', '코메디닷컴', '미디어오늘', '시사저널', '주간경향']

In [12]:
#본문 내용중 NNG, NNP만 추출하여 Noun 컬럼에 입력
for i in tqdm(range(len(data))):
    nouns = []
    a = posTag(data.loc[i,'content']) # 형태소 분석 코드
    #print(a)
    a_nouns = [n for n, tag in a if tag in ["SL","NNG","NNP"]] #명사만 추출
    
    #불용어 제거
    for w in a_nouns:
        if w not in stop_words:
            nouns.append(w)
            
    data.at[i,'noun'] = nouns #데이터프레임에 입력  
    noun_db = ' '.join(s for s in nouns)#리스트 문자열로 변경
    #db에 입력
    sql = """ 
          INSERT INTO esg_crawl(Noun) VALUES(%s)
    """
    cursor.execute(sql,noun_db)
    conn.commit()

  0%|          | 0/3472 [00:00<?, ?it/s]

In [44]:
data.head(10)

,title,url,press,date,content,noun
0,"올해는 ESG 정보공시 표준화 원년, 압박 더 거세진다",https://news.naver.com/main/read.naver?mode=LS...,머니투데이,2021.01.01.,() 산업·금융·법조계 ESG 전문가 대담]/사진제공=이미지투데이2020년은 E...,"[산업, 금융, 법조, ESG, 전문가, 대담, 사진, 제공, 이미지, ESG, 환..."
1,"서학개미 사로잡은 美 ESG ETF, 올해도 高高할까",https://news.naver.com/main/read.naver?mode=LS...,매일경제,2021.01.01.,() 작년 착한 투자에 30조 몰려인베스코솔라 年 급등美기업 IPO 열기도 뜨거...,"[작년, 투자, 려인, 베스코, 솔라, 年, 급등, 美, 기업, IPO, 로빈, 후..."
2,"[신년사]방문규 수은 행장 ""수출 6000억달러 선봉장될 것""",https://news.naver.com/main/read.naver?mode=LS...,뉴시스,2021.01.01.,() ESG 경영 강화...수출 중소중견기업 맞춤형 지원 방문규 수출입은행장 (사...,"[ESG, 경영, 강화, 수출, 중소, 중견, 기업, 맞춤, 지원, 방문, 규, 수..."
3,[韓산업 대전환 ①] 격변기의 시대…대전환 앞당겼다,https://news.naver.com/main/read.naver?mode=LS...,아이뉴스24,2021.01.01.,() 코로나 팬데믹에 뉴노멀 시대 활짝…대내외 경제 불안 속 미래 먹거리 찾기 총...,"[코로나, 믹, 뉴노, 멀, 시대, 대내외, 경제, 불안, 속, 미래, 먹거리, 총..."
4,"펀드사태 혼쭐난 금융권, 외부 `포청천` 영입",https://news.naver.com/main/read.naver?mode=LS...,매일경제,2021.01.01.,"() 4대 금융지주 인사 키워드는우리금융, 감사원 출신 신민철하나銀은 이인영 변호...","[금융, 지주, 인사, 키워드, 금융, 감사원, 출신, 신민철, 銀, 이인영, 변호..."
5,"[TF초점] ""쉴 틈 없다"" 2021년 4대 그룹 총수 앞에 놓인 과제는",https://news.naver.com/main/read.naver?mode=LS...,더팩트,2021.01.01.,() 이재용 삼성전자 부회장은 지난달 30일 서울고법 형사1부 심리로 진행된 국정...,"[이재용, 삼성전자, 부회장, 지난달, 서울, 고법, 형사, 심리, 진행, 국정, ..."
6,"[2021 DD 신년사] IT산업, 역동적으로 대한민국 이끌기를",https://news.naver.com/main/read.naver?mode=LS...,디지털데일리,2021.01.01.,() 2021년1월1일 오전 7시40분 울산 간절곶 해돋이 생중계 캡쳐존경하는 독...,"[오전, 울산, 간절곶, 해돋이, 생중, 캡쳐, 존경, 독자, 축년, 辛, 丑年, ..."
7,신임 농협은행장에 권준학 농협중앙회 상무,https://news.naver.com/main/read.naver?mode=LS...,국민일보,2021.01.01.,() 차기 농협은행장에 권준학(57·사진) 농협중앙회 기획조정본부장(상무)이 선임...,"[차기, 농협, 은행장, 권준, 학, 사진, 농협, 중앙, 기획, 조정, 본부, 장..."
8,[새해증시]'코스피 3000-코스닥1000' 올해 상반기 돌파한다,https://news.naver.com/main/read.naver?mode=LS...,머니투데이,2021.01.01.,"() 삼성전자 등 반도체 유망. 여유자금은 주식, 부동산, ESG 펀드에 ] 증...","[삼성전자, 반도체, 유망, 여유, 자금, 주식, 부동산, ESG, 펀드, 증시, ..."
9,새해 돈 있으면 부동산보다 주식…2배 이상 추천 많았다,https://news.naver.com/main/read.naver?mode=LS...,머니투데이,2021.01.01.,"() 달러, 채권 등 안전자산 인기 시들]금융투자업계 전문가들은 올해도 증시 강...","[채권, 안전, 자산, 인기, 시, 금융, 투자, 업계, 전문가, 올해, 증시, 강..."


In [45]:
data = data[['title','url','press','date','content','noun']]

In [46]:
data.to_csv('esg_dataset_2021_cleansing.csv', index = False, encoding = 'utf-8-sig')

  0%|          | 0/3472 [00:00<?, ?it/s]